## estimating Lift

In [1]:
arms = {
    'thrust': (-2.25 * u.inch, -0.125*u.inch),
    'weight': (0*u.inch,0*u.inch),
    'wing_ac': (-1*u.inch,1*u.inch),
    'stab_ac': (4*u.inch,-1*u.inch),
}

NameError: name 'u' is not defined

In [ ]:
forces = {
    'thrust': 0.045*u.grams,
    'weight': 2.06*u.grams,
    'lift_wing': 2.46*u.grams,
    'drag_wing': 0.03*u.grams,
    'lift_stab': -0.4*u.grams,
    'drag_stab': 0.015*u.grams
}

We can now write a simple function that determines the moment around the CG for given values of the various terms. To simplify this function, we will create a Python dictionary holding the values.

It should be apparent that all six force vectors need to be considered, since none of them (except weight) align with the center of gravity.

In [ ]:
def moment(a,f, debug=False):
    
    # thrust moment
    ta = a['thrust'][1] # y 
    tf = f['thrust']
    tm = ta * tf
    if debug:
        print("Thrust moment:",tm)
    # weight  - no moment, but we will calculate it anyway
    wa = a['weight'][0]
    wf = f['weight']
    wm = wa * wf
    if debug:
        print("Weight moment:",wm)
    
    # wing lift moment
    wla = a['wing_ac'][0]
    wlf = f['lift_wing']
    wlm = wla * wlf
    if debug:
        print("Wing lift moment:",wlm)
    
    # wing drag moment
    wda = a['wing_ac'][1]
    wdf = f['drag_wing']
    wdm = wda * wdf
    if debug:
        print("Wing drag moment:",wdm)    
    
    # stab lift moment
    sla = a['stab_ac'][0]
    slf = f['lift_stab']
    slm = sla * slf
    if debug:
        print("Stab lift moment:",slm)    
    
    # stab drag moment
    sda = a['stab_ac'][1]
    sdf = f['drag_stab']
    sdm = sda * sdf 
    if debug:
        print("Stab drag moment:",sdm)
        
    total_moment = tm + wm + wlm + wdm + slm + sdm 
    print('total moment =',total_moment)


In [ ]:
moment(arms, forces, debug=True)

By convention, a positive moment pitched the node up. So this one is going to fly about like many of my  first flights!

For stable level flight, the total moment needs to be zero. Obviously, I threw in random values here, and our goal is to produce all of these numbers from our design and a little help from aerodynamics theory.

The nice thing about using code to help us make these calculations is that we can run parametric tests and see results quickly, We will use that fact later.

## Aerodynamic forces

The forces generated by the wing and stab depend on the airfoils we use and the actual flight conditions. Since most builder's use simple circular arc airfoils, I went in search of data on common thicknesses. The Wart uses a 4% arc on the wing, and a 2% arc on the stab. Of course, I could not find any usable data for those airfoils, but I did find data for a 6% and a 3% arch airfoils that will at least let us begin this development.

The lift and drag forces will be calculated using the definitions of the lift and drag coefficients:
    
\begin{equation}
C_l = \frac{2L}{\rho u^2 S}
\end{equation}

or

\begin{equation}
L = \frac{1}{2}\rho u^2 C_l S
\end{equation}

In these equations, **L** and **D** are the lift forces, **S** is the surface area of the wing or stab, $\rho$ is the air density and **u** is the flight velocity. 

Similarly:

\begin{equation}
D = \frac{1}{2}\rho u^2 C_d S
\end{equation}

Formally, we should also include the moment coefficient, which accounts for the center of pressure not falling exactly at the aerodynamic center

\begin{equation}
M = \frac{1}{2}\rho u^2 C_n S c
\end{equation}

Where **c** is the mean chord.

For testing purposes, we will use these values for density and velocity:



In [ ]:
flight = {
    'density': 0.00231683 *u.slugs/u.ft**3,
    'dynamic_viscosity': 0.0000180338 * u.pascal * u.second,
    'velocity': 2.0 * u.mph
}
flight['kinematic_viscosity'] = flight['dynamic_viscosity']/flight['density']
flight['q'] = 0.5* (flight['density'] * flight['velocity'] ** 2)
flight['wing_re'] = flight['density'] * flight['velocity'] * wing['mean_chord'] / flight['dynamic_viscosity']
flight['stab_re'] = flight['density'] * flight['velocity'] * stab['mean_chord'] / flight['dynamic_viscosity']

print_dict(flight,"Atmosphere Properties")

In [ ]:
wing_re = flight['wing_re'].to_base_units()
wing_re

In [ ]:
stab_re = flight['stab_re'].to_base_units()
stab_re

Getting airfoil data for these Reynolds Numbers is not easy. McLean arbitrarily used a $C_lw$ of 1.0, and calculated the required $C_ls$ to balance moments. He used pitching moment coefficient values of -0.10 and -0.05 for the wing and stab.He did not include any other forces. 

Rather than just use these numbers, we will use the most basic theories of aerodynamics to estimate the coefficients. 

*Thin Airfoil Theory*, is often used as a first approximation for calculating lift and pitching moment data. If we use *Prandtl's Lifting Line Theory* we can get an approximation for the induced drag, and profile drag which together as the major sources of drag at low speeds.

we can estimate the $C_d$ values by examining typical lift/drag curves. 
